In [1]:
import sys
import time
from pathlib import Path

import numpy as np
import torch
from IPython.display import Markdown, display
from openvino.runtime import Core

from tools import load_policy,get_args
from model import DRL_GAT
args = get_args()

In [2]:
DIRECTORY_NAME = "model"
BASE_MODEL_NAME = DIRECTORY_NAME + "/DRL_GAT"

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored.
model_path = Path(BASE_MODEL_NAME).with_suffix(".pth")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")
print(model_path)

model/DRL_GAT.pth


In [3]:
url='pretrained_models/PCT_setting1.pt'
PCT_policy =  DRL_GAT(args)
PCT_policy =  PCT_policy.cpu().eval()
model = load_policy(url, PCT_policy)
print('Pre-train model loaded!', url)

model_path.parent.mkdir(exist_ok=True)
torch.save(model.state_dict(), str(model_path))
print(f"Model saved at {model_path}")

pretrained_models/PCT_setting1.pt
Loading pre-train upper model pretrained_models/PCT_setting1.pt
Pre-train model loaded! pretrained_models/PCT_setting1.pt
Model saved at model/DRL_GAT.pth


In [4]:
if not onnx_path.exists():
    dummy_input = (1+(torch.randn(1, 131, 9))/2).int().float()
    print(dummy_input)

    # For the Fastseg model, setting do_constant_folding to False is required
    # for PyTorch>1.5.1
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        opset_version=11,
        do_constant_folding=False,
    )
    print(f"ONNX model exported to {onnx_path}.")
else:
    print(f"ONNX model {onnx_path} already exists.")

tensor([[[1., 0., 0.,  ..., 1., 0., 2.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 1., 1.],
         [1., 0., 2.,  ..., 0., 0., 1.],
         [1., 0., 1.,  ..., 1., 1., 1.]]])
logP: tensor([[1.6056e-04, 3.6099e-05, 1.0016e-20, 1.0016e-20, 1.0016e-20, 1.1964e-04,
         1.0016e-20, 1.0016e-20, 1.0016e-20, 6.8880e-07, 1.0016e-20, 1.0016e-20,
         1.0016e-20, 1.0016e-20, 1.6056e-04, 8.8284e-06, 1.0016e-20, 4.8684e-05,
         1.0016e-20, 8.9976e-01, 1.0016e-20, 5.9626e-03, 1.0016e-20, 9.5347e-04,
         7.6036e-02, 1.0016e-20, 1.0016e-20, 2.2031e-04, 1.0016e-20, 1.0016e-20,
         2.1409e-04, 1.7933e-06, 3.0876e-05, 1.0016e-20, 1.0016e-20, 1.0016e-20,
         5.8546e-04, 1.0016e-20, 2.8143e-03, 4.1280e-06, 1.4123e-05, 1.7161e-03,
         1.0016e-20, 1.9240e-03, 1.0016e-20, 1.6056e-04, 1.6056e-04, 3.3933e-04,
         8.5213e-03, 5.0010e-05]], grad_fn=<DivBackward0>)


/root/workspace/Online-3D-BPP-PCT/graph_encoder.py:76: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  batch_size = int(q.size()[0] / graph_size)
/root/workspace/Online-3D-BPP-PCT/graph_encoder.py:80: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert input_dim == self.input_dim, "Wrong embedding dimension of input"
/root/workspace/Online-3D-BPP-PCT/graph_encoder.py:165: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constan

NameError: name 'dist' is not defined

In [5]:
# Construct the command for Model Optimizer.
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --input_shape "[1, 131, 9]"
                 --data_type FP16
                 --output_dir "{model_path.parent}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "model/DRL_GAT.onnx" --input_shape "[1, 131, 9]" --data_type FP16 --output_dir "model"`

In [6]:
if not ir_path.exists():
    print("Exporting ONNX model to IR... This may take a few minutes.")
    mo_result = %sx $mo_command
    print("\n".join(mo_result))
else:
    print(f"IR model {ir_path} already exists.")

Exporting ONNX model to IR... This may take a few minutes.
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/root/workspace/Online-3D-BPP-PCT/model/DRL_GAT.onnx
	- Path for generated IR: 	/root/workspace/Online-3D-BPP-PCT/model
	- IR output name: 	DRL_GAT
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1, 131, 9]
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- Force the usage of legacy Frontend of Model Optimizer for model 